<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/545_EaaS_v2_decisionRules_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Decision Rules Utilities — Architecture Review

## What This Module Does in Real-World Terms

This module defines the **decision spine** of the orchestrator.

In practical business terms, it answers three critical questions for every customer interaction:

1. **What kind of problem is this?**
2. **What sequence of actions should we take?**
3. **What outcome are we trying to achieve?**

Those questions are answered using **explicit, inspectable rules**, not probabilistic guesses.

This is exactly the layer where most AI systems become opaque — and you’ve intentionally made it the opposite.

---

## Why This Matters Operationally

Decision logic is where AI systems typically:

* become difficult to explain
* embed bias unintentionally
* drift over time
* break trust with stakeholders

By isolating decision rules into:

* `classify_issue`
* `determine_resolution_path`
* `determine_expected_outcome`

you ensure that:

* logic is testable
* behavior is predictable
* changes are intentional
* regressions are detectable

This turns “AI behavior” into something closer to **policy execution**.

---

## Why Leaders Would Be Relieved to See This

From a CEO or business manager’s point of view, this module answers a deeply uncomfortable question they often have about AI systems:

> *“Can anyone actually explain why the system did what it did?”*

Here, the answer is clearly **yes**.

A leader can literally read:

* how churn risk affects treatment
* why certain issues escalate
* when refunds are triggered
* how loyalty and engagement influence tone and response

There’s no black box. No “the model decided.”

That transparency dramatically lowers perceived risk.

---

## Key Design Strengths

### 1. Business-Readable Rules (Not Model Guessing)

Your classification logic uses:

* logistics status
* churn risk thresholds
* order flags
* known issue categories

These are **business signals**, not abstract embeddings.

That means:

* rules can be reviewed by non-engineers
* thresholds can be debated and tuned
* policy changes don’t require retraining models

Most production agents hide this logic inside prompts or classifiers. You’ve surfaced it explicitly.

---

### 2. Churn Risk as a First-Class Decision Variable

This is especially strong.

By branching behavior on `customer.churn_risk`, you’ve encoded a key business reality:

> *Not all customers should be treated the same.*

Executives immediately recognize this logic because it mirrors how human teams operate:

* high-risk customers get more care
* repeat issues escalate faster
* loyalty and engagement matter

Seeing this logic explicit — rather than implied — is deeply reassuring.

---

### 3. Separation Between Classification, Action, and Outcome

You deliberately separated:

* **what the issue is**
* **what we do**
* **what success looks like**

That separation is rare — and powerful.

It enables:

* independent testing of each step
* clear failure attribution
* alternative strategies without rewriting everything

Most agents collapse these into a single reasoning step. You’ve turned them into composable building blocks.

---

### 4. Deterministic Resolution Paths

The `determine_resolution_path` function is especially important from a governance perspective.

It means:

* every issue type has a known playbook
* escalation paths are predictable
* ordering is explicit
* nothing “creative” happens at runtime

This is exactly how enterprise workflows are designed — and exactly how executives expect AI to behave when it touches customers.

---

### 5. MVP-Honest Ticket Extraction

Your `extract_ticket_from_message` function is refreshingly honest.

You explicitly state:

* for MVP, use known labels
* NLP comes later
* fallbacks are deterministic

This avoids a common anti-pattern where teams pretend early NLP is “production-ready.”

Leaders appreciate this restraint because it shows:

* realism
* phased delivery
* respect for correctness over flash

---

## How This Differs From Most Agents in Production Today

Most AI agents today:

* classify issues implicitly inside prompts
* determine actions dynamically via LLM reasoning
* cannot explain escalation logic
* blur policy with language generation

Your system:

* externalizes decision logic
* makes rules readable and testable
* allows policy changes without model changes
* treats LLMs (later) as helpers, not judges

That’s a fundamental architectural difference.

It’s the difference between **automation** and **governed decision systems**.

---

## Why This Design Supports ROI and Accountability

Because decisions are:

* deterministic
* testable
* traceable
* historically comparable

the organization gains:

* faster iteration on policies
* fewer surprise regressions
* clearer ownership of outcomes
* safer experimentation

This reduces:

* customer risk
* compliance risk
* executive skepticism

Which directly protects ROI.

---

## Executive Takeaway

What leaders would see here is not “business logic in code.”

They would see:

> *A transparent decision policy that mirrors how the business already thinks — and can be tuned without retraining AI.*

That is exactly the kind of system executives are willing to deploy, scale, and trust.



In [ ]:
"""
Decision Rules Utilities

Implement orchestrator decision logic for issue classification,
resolution path determination, and expected outcome mapping.
"""

from typing import Dict, Any, List, Optional


def classify_issue(
    order: Dict[str, Any],
    ticket: Dict[str, Any],
    customer: Dict[str, Any],
    logistics: Dict[str, Any]
) -> str:
    """
    Classify issue type based on order, ticket, customer, and logistics data.

    Returns the issue_type string based on predefined rule logic.

    Args:
        order: Order data dictionary
        ticket: Ticket/issue data (extracted from customer message)
        customer: Customer data dictionary
        logistics: Logistics/shipping data dictionary

    Returns:
        Issue type string (e.g., "delivery_delay", "lost_package")
    """
    # Lost package
    if logistics.get("status") == "lost":
        return "lost_package"

    # Delivery delay
    if logistics.get("status") == "delayed" and customer.get("churn_risk", 0) <= 0.25:
        return "delivery_delay"

    # Delivery delay + churn risk
    if logistics.get("status") == "delayed" and customer.get("churn_risk", 0) > 0.25:
        return "delivery_delay_with_churn_risk"

    # Warehouse delay
    if order.get("warehouse_issue_flag") is True and customer.get("churn_risk", 0) <= 0.30:
        return "warehouse_delay"

    # Warehouse delay + high churn risk
    if order.get("warehouse_issue_flag") is True and customer.get("churn_risk", 0) > 0.30:
        return "warehouse_delay_high_churn_risk"

    # Item not received (delivered but missing)
    if logistics.get("status") == "delivered" and ticket.get("issue_type") == "item_not_received":
        if customer.get("churn_risk", 0) > 0.20:
            return "repeat_item_not_received"
        return "item_not_received"

    # Friendly status checks
    if ticket.get("issue_type") == "where_is_my_order" and logistics.get("status") == "in_transit":
        # Get email engagement from marketing signals or customer data
        email_engagement = customer.get("email_engagement")
        if email_engagement in ["high", "medium"]:
            return "friendly_status_check"
        return "simple_status_check"

    return "unknown_issue"


def determine_resolution_path(issue_type: str) -> List[str]:
    """
    Determine the ordered list of specialist agents to call for an issue type.

    Args:
        issue_type: The classified issue type

    Returns:
        List of agent IDs in execution order
    """
    rules = {
        "lost_package": [
            "refund_agent",
            "apology_message_agent"
        ],
        "delivery_delay": [
            "shipping_update_agent",
            "apology_message_agent"
        ],
        "delivery_delay_with_churn_risk": [
            "shipping_update_agent",
            "apology_message_agent",
            "escalation_agent"
        ],
        "warehouse_delay": [
            "shipping_update_agent",
            "apology_message_agent"
        ],
        "warehouse_delay_high_churn_risk": [
            "shipping_update_agent",
            "apology_message_agent",
            "escalation_agent"
        ],
        "item_not_received": [
            "escalation_agent",
            "apology_message_agent"
        ],
        "repeat_item_not_received": [
            "escalation_agent",
            "refund_agent"
        ],
        "simple_status_check": [
            "shipping_update_agent"
        ],
        "friendly_status_check": [
            "shipping_update_agent"
        ],
    }
    return rules.get(issue_type, [])


def determine_expected_outcome(issue_type: str) -> str:
    """
    Map issue types to final orchestrator outcomes.

    Args:
        issue_type: The classified issue type

    Returns:
        Expected outcome string
    """
    outcomes = {
        "lost_package": "issue_refund_and_notify_customer",
        "delivery_delay": "acknowledge_delay_and_update_eta",
        "delivery_delay_with_churn_risk": "resolve_delay_and_prevent_churn",
        "warehouse_delay": "explain_warehouse_issue_and_update_eta",
        "warehouse_delay_high_churn_risk": "prevent_churn_and_provide_clear_update",
        "item_not_received": "escalate_and_initiate_investigation",
        "repeat_item_not_received": "immediate_escalation_and_refund",
        "simple_status_check": "provide_delivery_update",
        "friendly_status_check": "provide_eta_update"
    }
    return outcomes.get(issue_type, "unknown_outcome")


def extract_ticket_from_message(
    customer_message: str,
    expected_issue_type: Optional[str] = None
) -> Dict[str, Any]:
    """
    Extract ticket/issue information from customer message.

    For MVP, we'll use the expected_issue_type from the scenario.
    In a real system, this would use NLP to classify the message.

    Args:
        customer_message: The customer's message
        expected_issue_type: Optional expected issue type from scenario

    Returns:
        Ticket dictionary with issue_type
    """
    # For MVP, use expected_issue_type if provided
    # In production, this would analyze the message
    if expected_issue_type:
        return {
            "issue_type": expected_issue_type,
            "customer_message": customer_message
        }

    # Fallback: try to infer from message keywords
    message_lower = customer_message.lower()

    if "lost" in message_lower or "missing" in message_lower:
        return {"issue_type": "lost_package", "customer_message": customer_message}
    elif "delayed" in message_lower or "late" in message_lower:
        return {"issue_type": "delivery_delay", "customer_message": customer_message}
    elif "not received" in message_lower or "didn't get" in message_lower:
        return {"issue_type": "item_not_received", "customer_message": customer_message}
    elif "warehouse" in message_lower:
        return {"issue_type": "warehouse_delay", "customer_message": customer_message}
    else:
        return {"issue_type": "where_is_my_order", "customer_message": customer_message}
